In [1]:
import pandas as pd
usuarios = pd.read_csv('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/user.csv', usecols = ["user_id","review_count","fans"])
reviews = pd.read_csv('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/review.csv', usecols = ["review_id","text","user_id"])

In [2]:
merge = pd.merge(usuarios,reviews, on = "user_id")
merge = merge[merge.review_count > 200]

In [3]:
group = merge.groupby(["user_id","fans"])[["text"]].agg(list)

In [4]:
nuevo_df = pd.DataFrame(group)
nuevo_df = nuevo_df.reset_index()

In [5]:
textos = []
for texts in nuevo_df.text:
    texto_concat = ""
    for text in texts:
        texto_concat = texto_concat + text
    textos.append(texto_concat)

In [6]:
nuevo_df["textos_concatenados"] = textos

In [7]:
import zstandard
import gzip

def compress_size(bytefile):
    return len(gzip.compress((bytefile.encode()),compresslevel = 1))


def ncd(st1,st2):
    len1 = compress_size(st1)
    len2 = compress_size(st2)
    concat_len =  compress_size(st1+st2)
    return (concat_len - min(len1,len2))/max(len1,len2)

In [8]:
nuevo_df["compresion"] = nuevo_df.textos_concatenados.apply(lambda x: compress_size(x))
nuevo_df["tamaño_real"] = nuevo_df.textos_concatenados.apply(lambda x: len(x))

In [9]:
nuevo_df["compresion_ratio"] = nuevo_df["tamaño_real"]/nuevo_df["compresion"]

In [11]:
correlacion = nuevo_df[["compresion_ratio","fans"]]

In [12]:
correlacion.corr()

,compresion_ratio,fans
compresion_ratio,1.000000,-0.171608
fans,-0.171608,1.000000
